In [1]:
import polars as pl
import os
import numpy as np
import torch, torchvision
from torchvision.utils import make_grid
from glio.loaders import nifti
from glio.python_tools import get_all_files
from glio.jupyter_tools import clean_mem
from glio.visualize import Visualizer
import matplotlib.pyplot as plt
path = r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM"

In [2]:
acquisition = pl.read_csv(f"{path}/UPENN-GBM_acquisition.csv", infer_schema_length=10000)
acquisition

ID,Manufacturer,Model,Magnetic Field Strength,T1 Imaging Frequency,T1 Repetition Time,T1 Echo Time,T1 Inversion Time,T1 Flip Angle,T1 Pixel Spacing,T1 Slice Thickness,T1GD Imaging Frequency,T1GD Repetition Time,T1GD Echo Time,T1GD Inversion Time,T1GD Flip Angle,T1GD Pixel Spacing,T1GD Slice Thickness,T2 Imaging Frequency,T2 Repetition Time,T2 Echo Time,T2 Flip Angle,T2 Pixel Spacing,T2 Slice Thickness,FLAIR Imaging Frequency,FLAIR Repetition Time,FLAIR Echo Time,FLAIR Inversion Time,FLAIR Flip Angle,FLAIR Pixel Spacing,FLAIR Slice Thickness,DTI Imaging Frequency,DTI Repetition Time,DTI Echo Time,DTI Flip Angle,DTI Pixel Spacing,DTI Slice Thickness,DSC Imaging Frequency,DSC Repetition Time,DSC Echo Time,DSC Flip Angle,DSC Pixel Spacing,DSC Slice Thickness
str,str,str,f64,f64,f64,f64,i64,i64,str,f64,f64,f64,f64,i64,i64,str,f64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,i64,str,i64,f64,i64,i64,i64,str,f64,f64,i64,f64,i64,str,f64
"""UPENN-GBM-0000…","""SIEMENS""","""TrioTim""",3.0,123.256776,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256799,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256698,3200.0,458.0,120,"""0.896875023841…",0.9,123.256693,9420,140.0,2500.0,170,"""0.9375\0.9375""",3,123.256673,5800,83,90,"""2.34375\2.3437…",3.0,123.256697,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0000…","""SIEMENS""","""TrioTim""",3.0,123.256765,1760.0,3.87,950,15,"""0.9765625\0.97…",1.0,123.256802,1760.0,3.87,950,15,"""0.9765625\0.97…",1.0,123.256699,3200.0,444.0,120,"""0.896875\0.896…",0.9,123.256687,9420,140.0,2500.0,170,"""0.9375\0.9375""",3,123.256645,4200,83,90,"""1.71875\1.7187…",3.0,123.256628,2000,45.0,90,"""1.6875\1.6875""",3.0
"""UPENN-GBM-0000…","""SIEMENS""","""TrioTim""",3.0,123.256642,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256642,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256651,3200.0,458.0,120,"""0.896875023841…",0.9,123.256665,9420,141.0,2500.0,170,"""0.9375\0.9375""",3,123.256638,5400,91,90,"""1.71875\1.7187…",3.0,123.256655,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0000…","""SIEMENS""","""TrioTim""",3.0,123.256789,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256789,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256719,3200.0,458.0,120,"""0.896875023841…",0.9,123.256744,9420,141.0,2500.0,156,"""0.9375\0.9375""",3,123.256702,4900,83,90,"""1.71875\1.7187…",3.0,123.256744,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0000…","""SIEMENS""","""TrioTim""",3.0,123.256727,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256779,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256675,3200.0,458.0,120,"""0.896875023841…",0.9,123.256687,9420,140.0,2500.0,170,"""0.9375\0.9375""",3,123.256661,4900,83,90,"""1.71875\1.7187…",3.0,123.256677,2000,45.0,90,"""1.71875\1.7187…",3.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UPENN-GBM-0062…","""SIEMENS""","""TrioTim""",3.0,123.256467,1760.0,3.11,950,15,"""0.9765625\0.97…",1.0,123.256467,1760.0,3.11,950,15,"""0.9765625\0.97…",1.0,123.256467,4680.0,85.0,160,"""0.9375\0.9375""",3.0,123.256451,9420,141.0,2500.0,170,"""0.9375\0.9375""",3,123.256398,5000,86,90,"""1.71875\1.7187…",3.0,123.256457,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0062…","""SIEMENS""","""TrioTim""",3.0,123.257151,1760.0,3.11,950,15,"""0.9765625\0.97…",1.0,123.257151,1760.0,3.11,950,15,"""0.9765625\0.97…",1.0,123.257151,4680.0,85.0,160,"""0.9375\0.9375""",3.0,123.25714,9420,141.0,2500.0,170,"""0.9375\0.9375""",3,123.257104,5000,86,90,"""1.71875\1.7187…",3.0,123.257125,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0062…","""SIEMENS""","""Verio""",3.0,123.261026,1760.0,3.15,950,15,"""0.48828125\0.4…",1.0,123.261026,1760.0,3.15,950,15,"""0.48828125\0.4…",1.0,123.26104,5410.0,86.0,160,"""0.9375\0.9375""",3.0,123.261131,9420,111.0,2550.0,170,"""0.46875\0.4687…",3,123.26105,6300,100,90,"""1.71875\1.7187…",3.0,123.261118,2000,45.0,90,"""1.71875\1.7187…",4.0


In [3]:
clinical_data = pl.read_csv(f"{path}/UPENN-GBM_clinical_info_v2.1.csv", infer_schema_length=10000)
clinical_data

ID,Gender,Age_at_scan_years,Survival_from_surgery_days_UPDATED,Survival_Status,Survival_Censor,IDH1,MGMT,KPS,GTR_over90percent,Time_since_baseline_preop,PsP_TP_score
str,str,f64,str,str,str,str,str,str,str,str,str
"""UPENN-GBM-0000…","""F""",52.16,"""960""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA"""
"""UPENN-GBM-0000…","""F""",61.3,"""291""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA"""
"""UPENN-GBM-0000…","""M""",42.82,"""2838""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA"""
"""UPENN-GBM-0000…","""M""",33.43,"""623""","""Deceased""","""Not Available""","""NOS/NEC""","""Not Available""","""Not Available""","""Y""","""0""","""NA"""
"""UPENN-GBM-0000…","""M""",53.33,"""1143""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA"""
…,…,…,…,…,…,…,…,…,…,…,…
"""UPENN-GBM-0062…","""F""",57.19,"""288""","""Deceased""","""Not Available""","""Wildtype""","""Unmethylated""","""Not Available""","""Not Applicable…","""234""","""5"""
"""UPENN-GBM-0062…","""F""",57.4,"""365""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Not Applicable…","""Not Available""","""4"""
"""UPENN-GBM-0062…","""M""",55.09,"""79""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Not Applicable…","""Not Available""","""6"""


In [4]:
data_availability = pl.read_csv(f"{path}/UPENN-GBM_data_availability.csv", infer_schema_length=10000)
data_availability

ID,Structural imaging,DTI imaging,DSC imaging,Automatic Tumor Segmentation,Corrected Tumor Segmentation,Gender,Age,Overall Survival,IDH1,MGMT,KPS,Extent of Resection,Associated Baseline Scan,Associated Recurrence Scan,PsP_TP score,CaPTk Radiomic Features
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UPENN-GBM-0000…","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
"""UPENN-GBM-0000…","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
"""UPENN-GBM-0000…","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
"""UPENN-GBM-0000…","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""not available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
"""UPENN-GBM-0000…","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UPENN-GBM-0062…","""available""","""available""","""available""","""not available""","""not available""","""available""","""available""","""not available""","""available""","""available""","""not available""","""not available""","""not available""","""-""","""available""","""not available"""
"""UPENN-GBM-0062…","""available""","""available""","""available""","""not available""","""not available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""not available""","""not available""","""-""","""available""","""not available"""
"""UPENN-GBM-0062…","""available""","""available""","""available""","""not available""","""not available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""not available""","""not available""","""-""","""available""","""not available"""


In [5]:
radiomic_features_paths = get_all_files(f"{path}/radiomic_features_CaPTk")[1:]
radiomic_features = [pl.read_csv(path, infer_schema_length=10000).rename({"SubjectID": "ID"}) for path in radiomic_features_paths]

In [6]:
all_files = get_all_files(r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM\NIfTI-files", extensions=["nii.gz"])
id_length = len("UPENN-GBM-00001_11")
files_dict = {}

for file in all_files:
    filename = os.path.basename(file)
    study_id = filename[:id_length]
    file_modality = f"PATH{filename[id_length:]}"
    if study_id not in files_dict:
        files_dict[study_id] = {"ID": study_id}
    files_dict[study_id][file_modality] = file

files_dict = list(files_dict.values())

filepaths_df = pl.from_dicts(files_dict)
filepaths_df

ID,PATH_automated_approx_segm.nii.gz,PATH_DSC.nii.gz,PATH_DSC_ap-rCBV.nii.gz,PATH_DSC_PH.nii.gz,PATH_DSC_PSR.nii.gz,PATH_DTI_AD.nii.gz,PATH_DTI_FA.nii.gz,PATH_DTI_RD.nii.gz,PATH_DTI_TR.nii.gz,PATH_FLAIR.nii.gz,PATH_T1.nii.gz,PATH_T1GD.nii.gz,PATH_T2.nii.gz,PATH_FLAIR_unstripped.nii.gz,PATH_T1GD_unstripped.nii.gz,PATH_T1_unstripped.nii.gz,PATH_T2_unstripped.nii.gz,PATH_segm.nii.gz
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UPENN-GBM-0000…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
"""UPENN-GBM-0000…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…"
"""UPENN-GBM-0000…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
"""UPENN-GBM-0000…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
"""UPENN-GBM-0000…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UPENN-GBM-0062…",null,"""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
"""UPENN-GBM-0062…",null,"""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null
"""UPENN-GBM-0062…",null,"""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…","""E:\dataset\PKG…",null


In [7]:
combined_concat = pl.concat(radiomic_features + [acquisition, clinical_data, data_availability], how="align")
combined_concat

ID,DSC_ap-rCBV_ED_Intensity_CoefficientOfVariation,DSC_ap-rCBV_ED_Intensity_Energy,DSC_ap-rCBV_ED_Intensity_InterQuartileRange,DSC_ap-rCBV_ED_Intensity_Kurtosis,DSC_ap-rCBV_ED_Intensity_Maximum,DSC_ap-rCBV_ED_Intensity_Mean,DSC_ap-rCBV_ED_Intensity_MeanAbsoluteDeviation,DSC_ap-rCBV_ED_Intensity_Median,DSC_ap-rCBV_ED_Intensity_MedianAbsoluteDeviation,DSC_ap-rCBV_ED_Intensity_Minimum,DSC_ap-rCBV_ED_Intensity_Mode,DSC_ap-rCBV_ED_Intensity_NinetiethPercentile,DSC_ap-rCBV_ED_Intensity_QuartileCoefficientOfVariation,DSC_ap-rCBV_ED_Intensity_Range,DSC_ap-rCBV_ED_Intensity_RootMeanSquare,DSC_ap-rCBV_ED_Intensity_Skewness,DSC_ap-rCBV_ED_Intensity_StandardDeviation,DSC_ap-rCBV_ED_Intensity_Sum,DSC_ap-rCBV_ED_Intensity_TenthPercentile,DSC_ap-rCBV_ED_Intensity_Variance,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-10_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-10_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-11_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-11_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-12_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-12_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-13_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-13_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-14_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-14_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-15_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-15_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-1_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-1_Probability,…,DTI Echo Time,DTI Flip Angle,DTI Pixel Spacing,DTI Slice Thickness,DSC Imaging Frequency,DSC Repetition Time,DSC Echo Time,DSC Flip Angle,DSC Pixel Spacing,DSC Slice Thickness,Gender,Age_at_scan_years,Survival_from_surgery_days_UPDATED,Survival_Status,Survival_Censor,IDH1,MGMT,KPS,GTR_over90percent,Time_since_baseline_preop,PsP_TP_score,Structural imaging,DTI imaging,DSC imaging,Automatic Tumor Segmentation,Corrected Tumor Segmentation,Gender_PL_CONCAT_RIGHT,Age,Overall Survival,IDH1_PL_CONCAT_RIGHT,MGMT_PL_CONCAT_RIGHT,KPS_PL_CONCAT_RIGHT,Extent of Resection,Associated Baseline Scan,Associated Recurrence Scan,PsP_TP score,CaPTk Radiomic Features
str,f64,i64,i64,f64,i64,f64,f64,i64,f64,i64,i64,i64,f64,i64,f64,f64,f64,i64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,…,i64,i64,str,f64,f64,i64,f64,i64,str,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UPENN-GBM-0000…",0.780117,78795500,46,5.243152,255,47.649409,-9.9165e-13,35,12.649409,0,17,102,0.547619,255,60.43316,1.457342,37.172096,1028036,14,1381.764723,2940,0.136269,114,0.005284,75,0.003476,45,0.002086,21,0.000973,8,0.000371,14,0.000649,7027,0.325701,…,83,90,"""2.34375\2.3437…",3.0,123.256697,2000,45.0,90,"""1.71875\1.7187…",3.0,"""F""",52.16,"""960""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA""","""available""","""available""","""available""","""available""","""available""","""available""","""available""","""not available""","""available""","""not available""","""not available""","""available""","""-""","""not available""","""not available""","""available"""
"""UPENN-GBM-0000…",1.008908,247711011,28,8.722965,255,28.122532,7.1971e-12,19,9.122532,0,0,64,0.583333,255,39.948733,2.066103,28.373045,4365095,3,805.029691,65039,0.41902,311,0.002004,175,0.001127,84,0.000541,29,0.000187,20,0.000129,11,0.000071,42940,0.276645,…,83,90,"""1.71875\1.7187…",3.0,123.256628,2000,45.0,90,"""1.6875\1.6875""",3.0,"""F""",61.3,"""291""","""Deceased""","""Not Available""","""Wildtype""","""Not Available""","""Not Available""","""Y""","""0""","""NA""","""available""","""available""","""available""","""available""","""available""","""available""","""available"

In [8]:
cols = []
all_dfs_dict = {i:df for i, df in enumerate(radiomic_features + [filepaths_df, clinical_data, data_availability, acquisition])}
print(len(all_dfs_dict))
for i, df in enumerate(radiomic_features + [filepaths_df, clinical_data, data_availability, acquisition]):
    if df.columns in cols: del all_dfs_dict[i]
    cols.append(df.columns)
print(len(all_dfs_dict))
all_dfs = list(all_dfs_dict.values())

70
37


In [9]:
cols = []
for i, df in enumerate(all_dfs):
    if df.columns in cols: print(df.columns)
    cols.append(df.columns)

In [14]:
combined = all_dfs[0]
for df in all_dfs[1:]:
    combined = combined.join(df, on = "ID", how="outer_coalesce", join_nulls=True, validate="1:1")
combined

ID,DSC_ap-rCBV_ED_Intensity_CoefficientOfVariation,DSC_ap-rCBV_ED_Intensity_Energy,DSC_ap-rCBV_ED_Intensity_InterQuartileRange,DSC_ap-rCBV_ED_Intensity_Kurtosis,DSC_ap-rCBV_ED_Intensity_Maximum,DSC_ap-rCBV_ED_Intensity_Mean,DSC_ap-rCBV_ED_Intensity_MeanAbsoluteDeviation,DSC_ap-rCBV_ED_Intensity_Median,DSC_ap-rCBV_ED_Intensity_MedianAbsoluteDeviation,DSC_ap-rCBV_ED_Intensity_Minimum,DSC_ap-rCBV_ED_Intensity_Mode,DSC_ap-rCBV_ED_Intensity_NinetiethPercentile,DSC_ap-rCBV_ED_Intensity_QuartileCoefficientOfVariation,DSC_ap-rCBV_ED_Intensity_Range,DSC_ap-rCBV_ED_Intensity_RootMeanSquare,DSC_ap-rCBV_ED_Intensity_Skewness,DSC_ap-rCBV_ED_Intensity_StandardDeviation,DSC_ap-rCBV_ED_Intensity_Sum,DSC_ap-rCBV_ED_Intensity_TenthPercentile,DSC_ap-rCBV_ED_Intensity_Variance,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-10_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-10_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-11_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-11_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-12_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-12_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-13_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-13_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-14_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-14_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-15_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-15_Probability,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-1_Frequency,DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-1_Probability,…,T1 Echo Time,T1 Inversion Time,T1 Flip Angle,T1 Pixel Spacing,T1 Slice Thickness,T1GD Imaging Frequency,T1GD Repetition Time,T1GD Echo Time,T1GD Inversion Time,T1GD Flip Angle,T1GD Pixel Spacing,T1GD Slice Thickness,T2 Imaging Frequency,T2 Repetition Time,T2 Echo Time,T2 Flip Angle,T2 Pixel Spacing,T2 Slice Thickness,FLAIR Imaging Frequency,FLAIR Repetition Time,FLAIR Echo Time,FLAIR Inversion Time,FLAIR Flip Angle,FLAIR Pixel Spacing,FLAIR Slice Thickness,DTI Imaging Frequency,DTI Repetition Time,DTI Echo Time,DTI Flip Angle,DTI Pixel Spacing,DTI Slice Thickness,DSC Imaging Frequency,DSC Repetition Time,DSC Echo Time,DSC Flip Angle,DSC Pixel Spacing,DSC Slice Thickness
str,f64,i64,i64,f64,i64,f64,f64,i64,f64,i64,i64,i64,f64,i64,f64,f64,f64,i64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,…,f64,i64,i64,str,f64,f64,f64,f64,i64,i64,str,f64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,i64,str,i64,f64,i64,i64,i64,str,f64,f64,i64,f64,i64,str,f64
"""UPENN-GBM-0000…",0.780117,78795500,46,5.243152,255,47.649409,-9.9165e-13,35,12.649409,0,17,102,0.547619,255,60.43316,1.457342,37.172096,1028036,14,1381.764723,2940,0.136269,114,0.005284,75,0.003476,45,0.002086,21,0.000973,8,0.000371,14,0.000649,7027,0.325701,…,3.1,950,15,"""0.9765625\0.97…",1.0,123.256799,1760.0,3.1,950,15,"""0.9765625\0.97…",1.0,123.256698,3200.0,458.0,120,"""0.896875023841…",0.9,123.256693,9420,140.0,2500.0,170,"""0.9375\0.9375""",3,123.256673,5800,83,90,"""2.34375\2.3437…",3.0,123.256697,2000,45.0,90,"""1.71875\1.7187…",3.0
"""UPENN-GBM-0000…",1.008908,247711011,28,8.722965,255,28.122532,7.1971e-12,19,9.122532,0,0,64,0.583333,255,39.948733,2.066103,28.373045,4365095,3,805.029691,65039,0.41902,311,0.002004,175,0.001127,84,0.000541,29,0.000187,20,0.000129,11,0.000071,42940,0.276645,…,3.87,950,15,"""0.9765625\0.97…",1.0,123.256802,1760.0,3.87,950,15,"""0.9765625\0.97…",1.0,123.256699,3200.0,444.0,120,"""0.896875\0.896…",0.9,123.256687,9420,140.0,2500.0,170,"""0.9375\0.9375""",3,123.256645,4200,83,90,"""1.71875\1.7187…",3.0,123.256628,2000,45.0,90,"""1.6875\1.6875""",3.0
"""UPENN-GBM-0000…",1.03478,25529631,12,25.149419,255,15.956322,-7.1038e-13,11,4.956322,0,6,33,0.5,255,22.961297,3.674562,16.511286,772653,5,272.622581,33329,0.688289,10,0.000207,15,0.00031,3,0.000062,4,0.0

In [15]:
combined.columns

['ID',
 'DSC_ap-rCBV_ED_Intensity_CoefficientOfVariation',
 'DSC_ap-rCBV_ED_Intensity_Energy',
 'DSC_ap-rCBV_ED_Intensity_InterQuartileRange',
 'DSC_ap-rCBV_ED_Intensity_Kurtosis',
 'DSC_ap-rCBV_ED_Intensity_Maximum',
 'DSC_ap-rCBV_ED_Intensity_Mean',
 'DSC_ap-rCBV_ED_Intensity_MeanAbsoluteDeviation',
 'DSC_ap-rCBV_ED_Intensity_Median',
 'DSC_ap-rCBV_ED_Intensity_MedianAbsoluteDeviation',
 'DSC_ap-rCBV_ED_Intensity_Minimum',
 'DSC_ap-rCBV_ED_Intensity_Mode',
 'DSC_ap-rCBV_ED_Intensity_NinetiethPercentile',
 'DSC_ap-rCBV_ED_Intensity_QuartileCoefficientOfVariation',
 'DSC_ap-rCBV_ED_Intensity_Range',
 'DSC_ap-rCBV_ED_Intensity_RootMeanSquare',
 'DSC_ap-rCBV_ED_Intensity_Skewness',
 'DSC_ap-rCBV_ED_Intensity_StandardDeviation',
 'DSC_ap-rCBV_ED_Intensity_Sum',
 'DSC_ap-rCBV_ED_Intensity_TenthPercentile',
 'DSC_ap-rCBV_ED_Intensity_Variance',
 'DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Frequency',
 'DSC_ap-rCBV_ED_Histogram_Bins-16_Bins-16_Bin-0_Probability',
 'DSC_ap-rCBV_ED_Histogr

In [16]:
combined_arrow = combined.to_arrow()
combined_arrow.num_rows, combined_arrow.num_columns

(671, 4840)

In [20]:
import pyarrow.parquet as pq
pq.write_table(combined_arrow, "UPENN_GBM.parquet")